In [1]:
import lib.Mask2Former as m2f
import os

In [2]:
DATA_SOURCE = "combined"
DATA_LOCATION = "_data"
DATA_DIR = "coco"
os.environ["DETECTRON2_DATASETS"] = os.path.join(DATA_LOCATION, DATA_DIR)

# Convert the dataset to COCO format
The following commands convert the existing PNG mask-based dataset to the coco annotations required for training Mask2Former

In [ ]:
!cd {DATA_LOCATION} && python mask_to_coco.py --images {DATA_SOURCE}/val/images/ --masks {DATA_SOURCE}/val/leaf_instances/ --output {DATA_DIR}/annotations/instances_val2017.json --fixed-category-id 58 --fixed-category-name "potted plant"
!cd {DATA_LOCATION} && python mask_to_coco.py --images {DATA_SOURCE}/train/images/ --masks {DATA_SOURCE}/train/leaf_instances/ --output {DATA_DIR}/annotations/instances_train2017.json --fixed-category-id 58 --fixed-category-name "potted plant"

  File "/home/stefan.steinheber/bachelor/processing/leaf_segmentation/_data/mask_to_coco.py", line 141
    convert_masks_to_coco(images, masks, output, pool_size=pool_size, category=(fixed_category_id, fixed_category_name), default_categories)
                                                                                                                                           ^
SyntaxError: positional argument follows keyword argument
  File "/home/stefan.steinheber/bachelor/processing/leaf_segmentation/_data/mask_to_coco.py", line 141
    convert_masks_to_coco(images, masks, output, pool_size=pool_size, category=(fixed_category_id, fixed_category_name), default_categories)
                                                                                                                                           ^
SyntaxError: positional argument follows keyword argument


In [4]:
!cd {DATA_LOCATION} && mkdir {DATA_DIR}/train2017
!cd {DATA_LOCATION} && cp {DATA_SOURCE}/train/images/* {DATA_DIR}/train2017
!cd {DATA_LOCATION} && mkdir {DATA_DIR}/val2017
!cd {DATA_LOCATION} && cp {DATA_SOURCE}/val/images/* {DATA_DIR}/val2017

mkdir: cannot create directory ‘coco/train2017’: File exists


In [10]:
CONFIG = "configs/coco/instance-segmentation/swin/maskformer2_swin_base_IN21k_384_bs16_50ep.yaml"
NUM_GPUS = 1
BATCH_SIZE = 8
LEARNING_RATE = 0.001

In [11]:
!python lib/Mask2Former/train_net.py --config-file {CONFIG} --num-gpus {NUM_GPUS} SOLVER.IMS_PER_BATCH {BATCH_SIZE} SOLVER.BASE_LR {LEARNING_RATE}


Traceback (most recent call last):
  File "/home/stefan.steinheber/bachelor/processing/leaf_segmentation/lib/Mask2Former/train_net.py", line 23, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
